In [0]:
#from google.colab import drive 
#drive.mount('/content/gdrive') 

In [0]:
#%cd /content/gdrive/My\ Drive/Colab\ Notebooks

In [0]:
server = "stark-anchorage-45962.herokuapp.com"

In [20]:
!pip install newspaper3k

In [38]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bs4
from urllib.parse import quote

from newspaper import Article
import pandas as pd
import json
import re

import http.client

import warnings
warnings.filterwarnings("ignore")

import time


In [0]:
url_lst_all = ["https://economictimes.indiatimes.com/news/politics-and-nation/coronavirus-cases-in-india-live-news-latest-updates-april8/liveblog/75038326.cms", "https://www.pharmaceutical-technology.com/special-focus/covid-19/coronavirus-covid-19-outbreak-latest-information-news-and-updates/", "https://www.business-standard.com/article/current-affairs/coronavirus-live-updates-covid-19-cases-in-india-global-death-toll-state-wise-delhi-maharasthra-tablighi-nizamuddin-lockdown-extension-latest-news-120040800236_1.html", "https://timesofindia.indiatimes.com/india/coronavirus-india-cases-live-news-updates-lockdown-in-india-likely-to-be-extended/liveblog/75074630.cms"]

url_lst_statewise = ["https://timesofindia.indiatimes.com/india"]

url_lst_citywise = ["https://timesofindia.indiatimes.com/city"]

keywords = ["corona", "coronavirus", "lockdown", "covid-19", "covid19", "masks", "handwash", "mask", "infection", "staysafe", "quarantine", "stayathome", "pandemic", "shutdown"]

apikey = "c8c54678d99e43d9b11a09d74e8dfa51"


In [0]:
def connection(conn, link, author, source, text, location) :
    headers = {'Content-type': 'application/json'}
    data = {'url': str(link), 'author' : str(author), 'source' : str(source), 'text' : str(text), 'location' : str(location)}
    json_data = json.dumps(data)
    conn.request('POST', '/addPost', json_data, headers)
    response = conn.getresponse()
    print(response.read().decode())

In [0]:
def allNews() :
    articles = []
    urls = []
    #response = requests.get(url_lst_all[0], allow_redirects = True)
    #soup = BeautifulSoup(response.text)
    '''r = requests.get(url_lst_all[0]) 
    soup = BeautifulSoup(r.content, 'html5lib') 
    #text1 = soup.findAll('h3', class_ = "")
    text2 = soup.findAll('div', class_ = "blogSysn")
    #text3 = soup.findAll('span')
    for text in text2 :
        if(len(text.text) > 50):
            articles.append(text.text)
            
    r = requests.get(url_lst_all[1]) 
    soup = BeautifulSoup(r.content, 'html5lib') 
    #text1 = soup.findAll('h3')
    text2 = soup.findAll('a')
    text3 = soup.findAll('li', class_ = "row")
    #text4 = soup.findAll('div', class_ = "large-9 columns")
    for txt in text3:
        regex = re.compile(r'[\n\r\t]')
        s = regex.sub(" ", txt.text)
        articles.append(s)
    for text in text2 :
        flag = 0
        blog = ""
        for ch in text:
            if(isinstance(ch, bs4.element.NavigableString)) :
                blog = blog + ch
        if(len(blog) > 25) :
            articles.append(blog)'''
        

    r = requests.get(url_lst_all[2]) 
    soup = BeautifulSoup(r.content, 'html5lib') 
    tags = soup.findAll('a')
    i = 0
    for tag in tags:
        try :
            if(tag['target'] == "_blank") :
                if(i < 3) :
                    i = i + 1
                    continue
                else :
                    if(len(tag.text) > 50) :
                        my_str = tag.text
                        _RE_COMBINE_WHITESPACE = re.compile(r"(?a:\s+)")
                        _RE_STRIP_WHITESPACE = re.compile(r"(?a:^\s+|\s+$)")

                        my_str = _RE_COMBINE_WHITESPACE.sub(" ", my_str)
                        my_str = _RE_STRIP_WHITESPACE.sub("", my_str)
                        for word in keywords:
                          if word in my_str.lower():
                            articles.append(my_str)
                            g = "https://www.business-standard.com" + tag['href']
                            urls.append(g)
                            break
        except:
            continue
            
    url = ('http://newsapi.org/v2/top-headlines?'
       'country=in&'
       'apiKey=c8c54678d99e43d9b11a09d74e8dfa51')
    response = requests.get(url)
    data = response.json()
    for article in data['articles'] :
        for word in keywords:
            try :
                if word in article['content'].lower():
                    articles.append(article['description'])
                    urls.append(article['url'])
            except :
                continue
    
    r = requests.get(url_lst_all[3]) 
    soup = BeautifulSoup(r.text, 'html5lib') 
    tags = soup.findAll('script')
    data = json.loads(tags[1].text, strict=False)
    for t in data['liveBlogUpdate'] :
        for word in keywords:
            if word in t['headline'].lower() :
                articles.append(t['headline'])
                urls.append(t['url'])
                break
    
    return articles, urls


In [0]:
def citywise() :
    prefix = "https://timesofindia.indiatimes.com"
    covid_links = []
    #covid_citywise = []
    response = requests.get(url_lst_citywise[0], allow_redirects = True)
    soup = BeautifulSoup(response.text)
    one_a_tag = soup.findAll('a')
    #print(one_a_tag)
    no_cities = 67
    index = 0
    cities_lst = []
    for line in one_a_tag :
            if(index > no_cities) :
                break
            try:
                link = line['href']
                if(link[:6] == "/city/") :
                    cities_lst.append(link)
                    index = index + 1
            except:
                continue
    #print(cities_lst)
    articles_citywise = []
    i = 0
    for url in cities_lst:
        covid_citywise = []
        articles_lst = []
        url_ = prefix + url
        #print(url_)
        words = url_.split("/")
        #print(words[-1])
        #covid_citywise.append(words[-1])
        articles_lst.append(words[-1])
        length_ = len(url_)
        length = len(url)
        #print(url_)
        response = requests.get(url_, allow_redirects = True)
        soup = BeautifulSoup(response.text)
        one_a_tag = soup.findAll('a')
        for link in one_a_tag :
            try :
                if(link['href'][-4:] == ".cms") :
                    if(link['href'][:length] == url):
                        temp = prefix + link['href']
                        articles_lst.append(temp)
                    elif(link['href'][:length_] == url_):
                        articles_lst.append(link['href'])
            except :
                continue
        #for ele in articles_lst :
            #print(ele)
        #break
        articles_citywise.append(articles_lst)
        #i = i + 1
        covid_links = []
        locations = []
        title = []
        content = []
        urls = []
        authors = []
        sources = []
    #print(articles_citywise)
    #return
    #from newspaper import Article
    for articles_lst in articles_citywise :
        covid_citywise = []
        i = 0
        for link in articles_lst:
            #print(link)
            if(i == 0) :
                loc = link
                print(loc)
                i = i + 1
                continue
            try :
                article = Article(link)
                article.download()
                article.parse()
                article.nlp()
                for word in keywords:
                    if word in article.text.lower() :
                        locations.append(loc)
                        title.append(article.title)
                        content.append(article.summary)
                        urls.append(link)
                        if(len(article.authors) != 0):
                          authors.append(article.authors)
                        else:
                          authors.append("")
                        sources.append(article.source_url)
                        #print(link)
                        #print(article.title)
                        #print(article.summary)
                        break
            except :
                continue
    
        #break
        #covid_links.append(covid_citywise)
    data = dict()
    data["title"] = title
    data["content"] = content
    data["location"] = locations
    data['url'] = urls
    data['author'] = authors
    data['source'] = sources
    #with open("./data_scrapper/citywise.json", 'w') as fh:
    #    fh.write(json.dumps(data))
    return data

In [0]:
def statewise() :
    prefix = "https://timesofindia.indiatimes.com"
    #covid_links = []
    response = requests.get(url_lst_statewise[0], allow_redirects = True)
    soup = BeautifulSoup(response.text)
    data = soup.findAll('meta')
    links_lst =[]
    i = -1
    for one in data :
        try:
            if(one['itemprop'] == "url") :
                url = str(one['content']).split('/')
                if(url[-2] == "india") :
                    i = i + 1
                    if(i == 0) :
                        continue
                    else :
                        links_lst.append(str(one['content']))
        except:
            continue
    #print(links_lst)
    locations = []
    title = []
    content = []
    urls = []
    authors = []
    sources = []
    for link in links_lst :
        #print(link)
        #covid_statewise = []
        parser = link.split("/")
        #covid_statewise.append(parser[-1])
        loc = parser[-1]
        r = requests.get(link) 
        print(loc)
        soup = BeautifulSoup(r.content, 'html5lib') 
        #tags = soup.findAll('script')
        tags_ = soup.findAll('a')
        #i = 0
        for tag in tags_ :
            try :
                if(tag["hid"] is not None) :
                    #print(tag)
                    article = Article(prefix + tag['href'])
                    article.download()
                    article.parse()
                    article.nlp()
                    for word in keywords:
                        if word in article.text.lower() :
                            locations.append(loc)
                            title.append(article.title)
                            content.append(article.summary)
                            urls.append(prefix + tag['href'])
                            if(len(article.authors) != 0):
                              authors.append(article.authors)
                            else:
                              authors.append("")
                            sources.append(article.source_url)
                            #print(article.title)
                            #print(article.summary)
                            #print("yes")
                            break
                    
            except:
                continue
        #break          
    data = dict()
    data["title"] = title
    data["content"] = content
    data["location"] = locations
    data['url'] = urls
    data['author'] = authors
    data['source'] = sources
    #print(data)
    #with open("./data_scrapper/statewise.json", 'w') as fh:
    #    fh.write(json.dumps(data))
    return data

In [0]:
#all india news
def get_all_news() :
  articles, urls = allNews()
  conn = http.client.HTTPSConnection(server)
  #limit = 0
  i = 0
  for article in articles:
      connection(conn, urls[i], "", "", article, "india")
      i = i + 1
      #limit = limit + 1
      #if(limit == 20) :
      #    time.sleep(61)
      #    limit = 0
      #    conn = http.client.HTTPSConnection(server)
      #print(article)
      #print("")
      #break
  #convert set of strings into json
  data = dict()
  data['articles'] = articles
  data['url'] = urls
  #with open("./data_scrapper/all.json", 'w') as fh:
  #    fh.write(json.dumps(data))
  return


In [0]:
#statewise
def get_statewise_news() :
  data = statewise()
  #title = data["title"] 
  content = data["content"] 
  locations = data["location"]  
  urls = data['url'] 
  authors = data['author']  
  sources = data['source'] 
  l = len(urls)
  #limit = 0
  #print(data)
  print(l)
  conn = http.client.HTTPSConnection(server)
  for j in range(l) :
    connection(conn, urls[j], authors[j], sources[j], content[j], locations[j])
    #limit = limit + 1
    #if(limit == 20) :
    #    time.sleep(61)
    #    limit = 0
    #    conn = http.client.HTTPSConnection(server)
  return

In [0]:
#citywise
def get_citywise_news() :
  data = citywise()
  #title = data["title"] 
  content = data["content"] 
  locations = data["location"]  
  urls = data['url'] 
  authors = data['author']  
  sources = data['source'] 
  l = len(urls)
  #limit = 0
  conn = http.client.HTTPSConnection(server)
  for j in range(l) :
    connection(conn, urls[j], authors[j], sources[j], content[j], locations[j])
    #limit = limit + 1
    #if(limit == 20) :
    #    time.sleep(61)
    #    limit = 0
    #   conn = http.client.HTTPSConnection(server)
  return


In [0]:
def run() :
  get_all_news()
  get_statewise_news()
  get_citywise_news()
  return

In [0]:
while(1) :
  run()